In [5]:
%pip install d3rlpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import d3rlpy

# get Pendulum dataset
dataset, env = d3rlpy.datasets.get_pendulum()

# Setup Decision Transformer
dt = d3rlpy.algos.DecisionTransformerConfig().create(device="cuda:0")

2024-01-06 19:10.21 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-01-06 19:10.21 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-01-06 19:10.21 [info     ] Action size has been automatically determined. action_size=1


In [7]:
# offline training
dt.fit(
   dataset,
   n_steps=1000,
   n_steps_per_epoch=1000,
   eval_env=env,
   eval_target_return=0,  # specify target environment return
)

dt.save_model("./models/pendulum-dt.d3")

2024-01-06 19:10.22 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2024-01-06 19:10.22 [info     ] Directory is created at d3rlpy_logs/DecisionTransformer_20240106191022
2024-01-06 19:10.22 [debug    ] Building models...            
2024-01-06 19:10.22 [debug    ] Models have been built.       
2024-01-06 19:10.22 [info     ] Parameters                     params={'observation_shape': [3], 'action_size': 1, 'config': {'type': 'decision_transformer', 'params': {'batch_size': 64, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'context_size': 20, 'max_timestep': 1000, 'learning_rate': 0.00

Epoch 1/1:   0%|          | 0/1000 [00:00<?, ?it/s]

/home/drinf/anaconda3/envs/decision-transformers/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


2024-01-06 19:11.21 [info     ] DecisionTransformer_20240106191022: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.004731279611587524, 'time_algorithm_update': 0.040909732341766356, 'loss': 0.791054716527462, 'time_step': 0.04574949383735657, 'environment': -1045.5063269756808} step=1000
2024-01-06 19:11.21 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20240106191022/model_1000.d3


In [8]:
import gym
import numpy as np
from gym.wrappers import RecordVideo

# start virtual display
d3rlpy.notebook_utils.start_virtual_display()

# wrap RecordVideo wrapper
env = RecordVideo(gym.make("Pendulum-v1", render_mode="rgb_array"), './videos/video-pendulum')
#env = gym.make("Pendulum-v1", render_mode="rgb_array")

# wrap as stateful actor for interaction
actor = dt.as_stateful_wrapper(target_return=0)

# interaction
observation, reward = env.reset(), 0.0
observation = observation[0]
while True:
    action = actor.predict(observation, reward)
    observation, reward, done, truncated, _ = env.step(action)
    if done or truncated:
        break


d3rlpy.notebook_utils.render_video("./videos/video-pendulum/rl-video-episode-0.mp4")

/home/drinf/anaconda3/envs/decision-transformers/lib/python3.10/site-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /home/drinf/Desktop/PROJECTS/machine_learning/fib_postgraduate/GameMindsDT/experiments/d3rlpy/video-pendulum folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/home/drinf/anaconda3/envs/decision-transformers/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Moviepy - Building video /home/drinf/Desktop/PROJECTS/machine_learning/fib_postgraduate/GameMindsDT/experiments/d3rlpy/video-pendulum/rl-video-episode-0.mp4.
Moviepy - Writing video /home/drinf/Desktop/PROJECTS/machine_learning/fib_postgraduate/GameMindsDT/experiments/d3rlpy/video-pendulum/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/drinf/Desktop/PROJECTS/machine_learning/fib_postgraduate/GameMindsDT/experiments/d3rlpy/video-pendulum/rl-video-episode-0.mp4
